In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [5]:
data = requests.get(standings_url)

In [6]:
import mysql.connector

def openConnection():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1998.Ronc@",
        database="futebol_data"
    )
    return conn

def closeConnection(conn):
    conn.close()

In [7]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
rows = standings_table.find_all('tr')

In [20]:
#Inserindo as informações a primeira vez no banco
conn = openConnection()
cursor = conn.cursor()
for i, row in enumerate(rows[1:], start=1):  # Ignora o cabeçalho
    columns = row.find_all('td')
    column_data = [col.text for col in columns]
    column_data.append(i)
    sql = """
    INSERT INTO team (team_name, matches_played, win, loss, draw, goals_for, goals_against, goals_difference, points, position)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s ,%s)
    """
    team_name = column_data[0].strip()
    # Modifique os índices de column_data conforme as colunas da tabela
    data_to_insert = (
            team_name,  # Nome do time
            int(column_data[1]),  # Jogos disputados
            int(column_data[2]),  # Vitórias
            int(column_data[3]),  # Derrotas
            int(column_data[4]),  # Empates
            int(column_data[5]),  # gols marcados
            int(column_data[6]),  # gols sofridos
            int(column_data[7]),  # saldo de gols
            int(column_data[8]),  # pontos
            int(column_data[19])  # Posição do time
        )
    cursor.execute(sql, data_to_insert)
    print(column_data)
# Commit das alterações
conn.commit()
# Fechando a conexão
cursor.close()
closeConnection(conn)

[' Liverpool', '8', '7', '0', '1', '15', '3', '+12', '21', '2.63', '15.7', '6.1', '+9.6', '+1.19', 'L W W W W', '60,246', 'Mohamed Salah, Luis Díaz - 5', 'Alisson', '', 1]
[' Manchester City', '8', '6', '2', '0', '19', '9', '+10', '20', '2.50', '15.4', '8.5', '+6.9', '+0.86', 'W D D W W', '52,790', 'Erling Haaland - 10', 'Ederson', '', 2]
[' Arsenal', '8', '5', '2', '1', '15', '8', '+7', '17', '2.13', '13.5', '9.1', '+4.4', '+0.56', 'W D W W L', '60,304', 'Kai Havertz - 4', 'David Raya', '', 3]
[' Aston Villa', '8', '5', '2', '1', '15', '10', '+5', '17', '2.13', '11.4', '8.5', '+2.9', '+0.36', 'W W D D W', '41,455', 'Ollie Watkins - 5', 'Emiliano Martínez', '', 4]
[' Brighton', '8', '4', '3', '1', '14', '10', '+4', '15', '1.88', '11.8', '13.3', '-1.4', '-0.18', 'D D L W W', '34,010', 'Danny Welbeck - 5', 'Bart Verbruggen', '', 5]
[' Chelsea', '8', '4', '2', '2', '17', '10', '+7', '14', '1.75', '15.4', '9.7', '+5.6', '+0.70', 'W W W D L', '39,528', 'Cole Palmer - 6', 'Robert Sánchez', '

In [9]:
#Código para atualizar as informações dos times. OBS: SÓ UTILIZAR SE JÁ TIVER INSERIDO OS TIMES DENTRO DO BANCO.
conn = openConnection()
cursor = conn.cursor()
for i, row in enumerate(rows[1:], start=1):  # Ignora o cabeçalho
    columns = row.find_all('td')
    column_data = [col.text for col in columns]
    column_data.append(i)
    
    # SQL para atualizar os dados do time
    sql_update = """
    UPDATE team 
    SET matches_played = %s, win = %s, loss = %s, draw = %s, 
        goals_for = %s, goals_against = %s, goals_difference = %s, 
        points = %s, position = %s
    WHERE team_name = %s
    """
    team_name = column_data[0].strip()
    
    # Modifique os índices de column_data conforme as colunas da tabela
    data_to_update = (
        int(column_data[1]),  # Jogos disputados
        int(column_data[2]),  # Vitórias
        int(column_data[3]),  # Derrotas
        int(column_data[4]),  # Empates
        int(column_data[5]),  # Gols marcados
        int(column_data[6]),  # Gols sofridos
        int(column_data[7]),  # Saldo de gols
        int(column_data[8]),  # Pontos
        int(column_data[19]),  # Posição
        team_name  # Nome do time (critério para atualização)
    )
    
    cursor.execute(sql_update, data_to_update)

# Commit das alterações
conn.commit()

# Fechando a conexão
cursor.close()
closeConnection(conn)

In [8]:
#tornando os links em formato completo
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [9]:
#Função para traduzir sigla de nacionalidade
def translate_country(country):
    country_dic = {
        'NED': 'Holanda',
        'EGY': 'Egito',
        'ENG': 'Inglaterra',
        'FRA': 'França',
        'HUN': 'Hungria',
        'SCO': 'Escocia',
        'ARG': 'Argentina',
        'BRA': 'Brasil',
        'COL': 'Colombia',
        'IRL': 'Irlanda',
        'URU': 'Uruguai',
        'GRE': 'Grecia',
        'ITA': 'Italia',
        'CZE': 'Republica tcheca',
        'JPN': 'Japao',
        'POR': 'Portugal',
        'NIR': 'Irlanda do norte'
    }
    return country_dic.get(country, 'Não identificado')

In [10]:
#Função para traduzir sigla de posição
def translate_pos(pos):
    pos_dic = {
        'GK': 'Goleiro',
        'LF': 'Lateral esquerdo',
        'DF': 'Zagueiro',
        'RB': 'Lateral direito',
        'FB': 'Lateral',
        'CB': 'Zagueiro central',
        'DM': 'Volante',
        'CM': 'Meio-campista Central',
        'LM': 'Meio-campista Esquerdo',
        'RM': 'Meio-campista Direito',
        'WM': 'Meio-campista Aberto',
        'MF': 'Meio campo',
        'AM':  'Meio-campista Ofensivo',
        'LW': 'Ponta Esquerdo',
        'RW': 'Ponta Direito',
        'FW': 'Atacante',
    }
    return pos_dic.get(pos, 'Posição inválida')

In [11]:
# Função para garantir a conversão para inteiro, tratando erros
def safe_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return 0 

In [42]:
#Função para tratar nome do time
def fetch_team_name(team_name):
    team_dic = {
        'Liverpool': 'Liverpool',
        'Manchester City': 'Manchester City',
        'Arsenal': 'Arsenal',
        'Aston Villa': 'Aston Villa',
        'Brighton and Hove Albion': 'Brighton',
        'Brighton': 'Brighton',
        'Chelsea': 'Chelsea',
        'Tottenham Hotspur': 'Tottenham',
        'Tottenham': 'Tottenham',
        'Newcastle United': 'Newcastle Utd',
        'Newcastle Utd': 'Newcastle Utd',
        'Fulham': 'Fulham',
        'Bournemouth': 'Bournemouth',
        'Manchester United': 'Manchester Utd',
        'Manchester Utd': 'Manchester Utd',
        "Nottingham Forest": "Nott'ham Forest",
        "Nott'ham Forest": "Nott'ham Forest",
        'Brentford': 'Brentford',
        'Leicester City': 'Leicester City',
        'West Ham United': 'West Ham',
        'West Ham': 'West Ham',
        'Everton': 'Everton',
        'Ipswich Town': 'Ipswich Town',
        'Crystal Palace': 'Crystal Palace',
        'Southampton': 'Southampton',
        'Wolverhampton Wanderers': 'Wolves',
        'Wolves': 'Wolves'
    }
    return team_dic.get(team_name, 'Time inválido')



In [13]:
#Extração de informação dos players dos times pela primeira vez
conn = openConnection()
cursor = conn.cursor()
for team_url in team_urls:
    data = requests.get(team_url)
    players = pd.read_html(data.text, match="Standard Stats")[0]
    players.columns = players.columns.droplevel(0)
    players.columns = players.columns.get_level_values(-1) 
    players = players[:-2]
    players.columns = [f"{col}_{i}" if players.columns.duplicated()[i] else col for i, col in enumerate(players.columns)]
    team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
    sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
    cursor.execute(sql_team_id,(team_name,))
    team_id = cursor.fetchone()
    for index, row in players.iterrows():
        sql = """
        INSERT INTO Player (player_name, Team_id, country, position, age, matches_played, started, minutes, goals, assist, penaltys_made, penaltys_attempted, yellow_card, red_card)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        player_values = (
            row['Player'],  # Nome do jogador
            team_id[0],  # ID do time 
            translate_country(row['Nation'].split()[-1]),  # Traduzir nacionalidade
            translate_pos(row['Pos'].strip()),
            safe_int(row['Age'].split('-')[0]),  # Idade
            safe_int(row['MP']),  # Jogos disputados
            safe_int(row['Starts']),  # Começou jogos
            safe_int(row['Min']),  # Minutos jogados
            safe_int(row['Gls']),  # Gols
            safe_int(row['Ast']),  # Assistências
            safe_int(row['PK']) if 'PK' in row else 0,  # Penaltys convertidos em gol, 0 se não existir
            safe_int(row['PKatt']) if 'PKatt' in row else 0,  # Penaltys tentados, 0 se não existir
            safe_int(row['CrdY']) if 'CrdY' in row else 0,  # Cartões amarelos, 0 se não existir
            safe_int(row['CrdR']) if 'CrdR' in row else 0   # Cartões vermelhos, 0 se não existir
        )
        cursor.execute(sql, player_values)
    time.sleep(2)
conn.commit()
cursor.close()
closeConnection(conn)



C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\1771754444.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  players = pd.read_html(data.text, match="Standard Stats")[0]


KeyboardInterrupt: 

In [25]:
#Extração de informação dos players dos times e atualizando os valores
conn = openConnection()
cursor = conn.cursor()
for team_url in team_urls:
    data = requests.get(team_url)
    players = pd.read_html(data.text, match="Standard Stats")[0]
    players.columns = players.columns.droplevel(0)
    players.columns = players.columns.get_level_values(-1) 
    players = players[:-2]
    players.columns = [f"{col}_{i}" if players.columns.duplicated()[i] else col for i, col in enumerate(players.columns)]
    team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
    sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
    cursor.execute(sql_team_id,(team_name,))
    team_id = cursor.fetchone()
    for index, row in players.iterrows():
        sql_update = """
            UPDATE Player 
            SET 
                country = %s, 
                position = %s, 
                age = %s, 
                matches_played = %s, 
                started = %s, 
                minutes = %s, 
                goals = %s, 
                assist = %s, 
                penaltys_made = %s, 
                penaltys_attempted = %s, 
                yellow_card = %s, 
                red_card = %s 
            WHERE player_name = %s
            """
        player_values = (
            translate_country(row['Nation'].split()[-1]),  # Traduzir nacionalidade
            translate_pos(row['Pos'].strip()),
            safe_int(row['Age'].split('-')[0]),  # Idade
            safe_int(row['MP']),  # Jogos disputados
            safe_int(row['Starts']),  # Começou jogos
            safe_int(row['Min']),  # Minutos jogados
            safe_int(row['Gls']),  # Gols
            safe_int(row['Ast']),  # Assistências
            safe_int(row['PK']) if 'PK' in row else 0,  # Penaltys convertidos em gol, 0 se não existir
            safe_int(row['PKatt']) if 'PKatt' in row else 0,  # Penaltys tentados, 0 se não existir
            safe_int(row['CrdY']) if 'CrdY' in row else 0,  # Cartões amarelos, 0 se não existir
            safe_int(row['CrdR']) if 'CrdR' in row else 0,   # Cartões vermelhos, 0 se não existir
            row['Player'],  # Nome do jogador
        )
        cursor.execute(sql_update, player_values)
    time.sleep(2)
conn.commit()
cursor.close()
closeConnection(conn)

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_21500\966906929.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  players = pd.read_html(data.text, match="Standard Stats")[0]


KeyboardInterrupt: 

In [14]:
# Função tradução dos dias da semana
def translate_day(day_code):
    week_days = {
        'Mon': 'Segunda-feira',
        'Tue': 'Terça-feira',
        'Wed': 'Quarta-feira',
        'Thu': 'Quinta-feira',
        'Fri': 'Sexta-feira',
        'Sat': 'Sábado',
        'Sun': 'Domingo'
    } 
    return week_days.get(day_code, 'Sigla inválida')

In [15]:
#Função de tradução do resultado da partida
def translate_result(result):
    results = {
        'W': 'Vitória',
        'D': 'Empate',
        'L': 'Derrota'
    }
    return results.get(result, 'Sigla inválida')

In [ ]:
#pegando informação das partidas dos times na premier league a primeira vez
conn = openConnection()
cursor = conn.cursor(buffered=True)
matches_array = []
for team_url in team_urls:
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    # Filtra apenas as partidas da Premier League
    premier_league_matches = matches[matches['Comp'] == 'Premier League']
    # Itera sobre as partidas e adiciona ao array
    for index, row in premier_league_matches.iterrows():
        venue = row['Venue']
        opponent_name = fetch_team_name(row['Opponent'])
        captain_name = row['Captain']
        if venue == 'Home':
            venue = 'Casa'
        if venue == 'Away':
            venue = 'Fora'
        # Consulta SQL para buscar o ID do oponente
        sql_opponent_id = "SELECT id FROM team WHERE team_name = %s"
        cursor.execute(sql_opponent_id, (opponent_name,))
        opponent_id = cursor.fetchone()  # Retorna o ID do oponente
        sql_captain_id = "SELECT id FROM player WHERE player_name = %s"
        captain_id = None
        if(type(captain_name) == str):
            cursor.execute(sql_captain_id, (captain_name,))
            captain_id = cursor.fetchone()[0]
        week_day = None
        week_day = translate_day(row['Day'].strip())
        result = None
        if(type(row["Result"]) == str):
            result = translate_result(row['Result'].strip())
        goals_for = row['GF']
        if(pd.isna(row['GF'])):
            goals_for = None
        goals_against = row['GA']
        if(pd.isna(row['GA'])):
            goals_against = None
        poss = row['Poss']
        if(pd.isna(row['Poss'])):
            poss = None
        formation = None
        if(type(row['Formation']) == str):
            formation = row['Formation']
        opp_formation = None
        if(type(row['Opp Formation']) == str):
            opp_formation = row['Opp Formation']
        team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
        sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
        cursor.execute(sql_team_id,(team_name,))
        team_id = cursor.fetchone()
        if opponent_id:
            opponent_id = opponent_id[0] 
        match_info = (
            row['Date'],            # Data da partida
            row['Round'],           #Rodada
            week_day,               #dia da semana
            venue,                  #Local da partida (casa/fora)
            result,                 #Resultado da partida
            goals_for,              #Gols marcados
            goals_against,              #Gols sofridos
            opponent_id,            #Adversário
            poss,            #Posse de bola
            captain_id,               #Capitão do time
            formation,       #Formação do time
            opp_formation,    #Formação do time adversário
            team_id[0]
        )
        sql = """
        INSERT INTO Matches (date, round, week_day, venue, result, goals_for, goals_against, opponent_team_id, poss, captain_id, formation, opponent_formation, team_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql ,match_info)
        matches_array.append(match_info)
    time.sleep(5)
conn.commit()
cursor.close()
closeConnection(conn)
matches_array